In [31]:
import numpy as np
from sklearn.metrics import euclidean_distances
import pandas as pd
import random
import sys
sys.path.insert(0, '../SOM-LVQ')
import LVQ

In [32]:
data = pd.read_csv(r"../data/AReM/Dataset1(combined).csv")
x = data.iloc[:, 0:6].values
y = data.iloc[:, 6].values
y[y==7] = 0
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)

# use Minmaxscaler because we use euclidean distance
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
x_train = minmax.fit_transform(x_train)
x_test = minmax.transform(x_test)

In [33]:
x_train_D1, _, y_train_D1, _ = train_test_split(x_train, y_train, test_size=0.1, random_state=42)
lvq = LVQ.LVQ(x=x_train, y=y_train, n_classes=7, n_neurons=90, epsilon=0.9, p_vectors=[], epsilon_dec_factor=0.001)

In [34]:
lvq.fit(x_train, y_train)

In [35]:
predicted_y = [lvq.predict(instance) for instance in x_test]

from sklearn.metrics import classification_report

print (classification_report(y_test, predicted_y, target_names=['0', '1', '2', '3', '4', '5', '6']))

             precision    recall  f1-score   support

          0       0.70      0.64      0.67       238
          1       1.00      1.00      1.00       248
          2       0.98      0.96      0.97       241
          3       0.65      0.75      0.70       242
          4       0.98      0.99      0.99       248
          5       0.87      0.79      0.83       226
          6       0.86      0.87      0.87       237

avg / total       0.86      0.86      0.86      1680



In [36]:
D1 = len(x_train_D1)

In [37]:
D2 = []
for i in range(D1):
    if(random.uniform(0, 1) < 0.5):
        while (1): 
            index = np.random.randint(0, len(x_train))
            if (lvq.predict(x_train[index]) == y_train[index]):
                continue
            else:
                D2.append(index)
                break
    else:
        while (1): 
            index = np.random.randint(0, len(x_train))
            if (lvq.predict(x_train[index]) != y_train[index]):
                continue
            else:
                D2.append(index)
                break

In [38]:
lvq_d2 = LVQ.LVQ(x=x_train[D2], y=y_train[D2], n_classes=7, n_neurons=90, epsilon=0.9, p_vectors=[], epsilon_dec_factor=0.001)

In [39]:
lvq_d2.fit(x_train[D2], y_train[D2])

In [40]:
predicted_y = [lvq_d2.predict(instance) for instance in x_test]
print (classification_report(y_test, predicted_y, target_names=['0', '1', '2', '3', '4', '5', '6']))

             precision    recall  f1-score   support

          0       0.58      0.62      0.60       238
          1       1.00      1.00      1.00       248
          2       0.96      0.97      0.96       241
          3       0.60      0.53      0.56       242
          4       0.98      0.99      0.99       248
          5       0.77      0.76      0.77       226
          6       0.79      0.83      0.81       237

avg / total       0.81      0.82      0.81      1680



In [41]:
D3 = []
train_len = len(x_train)
for i in range(train_len):
    if (lvq.predict(x_train[i]) == lvq_d2.predict(x_train[i])):
        continue
    else:
        D3.append(i)

In [42]:
lvq_d3 = LVQ.LVQ(x=x_train[D3], y=y_train[D3], n_classes=7, n_neurons=90, epsilon=0.9, p_vectors=[], epsilon_dec_factor=0.001)
tmp = lvq_d3.fit(x_train[D3], y_train[D3])

In [43]:
def predict(x):
    if (lvq.predict(x) == lvq_d2.predict(x)):
        return lvq.predict(x)
    else:
        return lvq_d3.predict(x)

In [44]:
predicted_y = [predict(instance) for instance in x_test]
print (classification_report(y_test, predicted_y, target_names=['0', '1', '2', '3', '4', '5', '6']))

             precision    recall  f1-score   support

          0       0.74      0.71      0.73       238
          1       1.00      1.00      1.00       248
          2       0.97      0.98      0.97       241
          3       0.70      0.74      0.72       242
          4       1.00      0.99      0.99       248
          5       0.88      0.81      0.85       226
          6       0.84      0.89      0.86       237

avg / total       0.88      0.88      0.88      1680

